In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['figure.figsize']=(7,7)
plt.rcParams.update({'font.size': 11})
# %config InlineBackend.close_figures=False # keep figures open in pyplot

## Plotting the function
$$ y\ =\ \frac{1}{15}\sum_{k=0}^{n}\left(-1\right)^{k}\left(\frac{\left(2n+1\right)!}{\left(2n-2k\right)!\left(2k+1\right)!}\right)\left(1-x^{2}\right)^{\left(n-k\right)}\cdot x^{2k} $$

In [ ]:
a = (1/15)*((-1)^k)*(math.factorial(2*n + 1)/(math.factorial(2*k + 1)*math.factorial(-1*2*k + 2*n)))